  \begin{equation}\label{eq:LVln}
    \begin{gathered}
      \dot{x}_{1}=x_{1}(3-x_{2}-x_{3}-x_{4})\,,\quad
      \dot{x}_{2}=x_{2}(x_{1}-x_{3})\,,\\
      \dot{x}_{3}=x_{3}(-1+x_{1}+x_{2}-x_{4})\,,\quad
      \dot{x}_{4}=x_{4}(-2+x_{1}+x_{3})\,,
    \end{gathered}
  \end{equation}
  where $x_5=\ln(x_1), x_6=\ln(x_2), x_7=\ln(x_3), x_8=\ln(x_4)$

In [2]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff

In [3]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff
output_directory = r'../../../Data/4D_Lotka_Volterra/4D_Lotka_T5_N10'
def generate_random_values():
    x1 = random.uniform(0.5, 4)
    x2 = random.uniform(0.3, 4)
    x3 = random.uniform(0.2, 4)
    x4 = random.uniform(0.4, 4)
    return x1, x2, x3, x4
# def calculate_c(x1, x2, x3):
#     c = x1 + x2 + x3
#     return c
def generate_random_values_based_on_c():
    x1 = random.uniform(0.5, 4)
    x2 = random.uniform(0.3, 4)
    x3 = random.uniform(0.2, 4)
    x4 = random.uniform(0.4, 4)
    return x1, x2, x3, x4
def generate_data(initial_conditions):
    def normalize(vector):
        norm = np.linalg.norm(vector)
        if norm == 0: 
            return vector
        return vector / norm
    def normalized_system(y, t):
        x1, x2, x3, x4 = y
        f = np.array([x1 * (3 - x2 - x3 - x4), x2 * (x1 - x3), x3 * (-1 + x1 + x2 - x4), x4 * (-2 + x1 + x3)])
        normalized_f = normalize(f)
        return normalized_f
    def compute_energy(x1, x2, x3, x4):
        return 0.5 * (x3**2 + x4**2) + 0.5 * (x1**2 + x2**2) + x1**2 * x2 - (1/3) * x2**3
    num_trajectories = 5
    t = np.linspace(0, 100, 10) # 10 data points per trajectory
    all_trajectory_data = []
    initial_conditions_to_print = []
    print("Initial data (x1, x2, x3, x4):")
    for i, initial_condition in enumerate(initial_conditions):
        print(f"({initial_condition[0]}, {initial_condition[1]}, {initial_condition[2]}, {initial_condition[3]})")
        sol = odeint(normalized_system, initial_condition, t)
        all_trajectory_data.append(sol)
    num_variables = 8 # Adjust number of variables that we need for the regression accordingly
    column_names = [f'x{i+1}' for i in range(num_variables)]
    column_names.append('trajectory')
    with open('50.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        for r, data in enumerate(all_trajectory_data):
            for j in range(len(t)):
                x1 = data[j, 0]
                x2 = data[j, 1]
                x3 = data[j, 2]
                x4 = data[j, 3]
                writer.writerow([x1, x2, x3, x4, math.log(x1), math.log(x2), math.log(x3), math.log(x4), r+1]) 
    output_directory1 = r'../../../results/4D_Lotka_Volterra/4D_Lotka_T5_N10'
    plt.figure(figsize=(10, 6))
    for i, sol in enumerate(all_trajectory_data):
        for j in range(sol.shape[1]):
            plt.plot(t, sol[:, j])
    plt.savefig(os.path.join(output_directory1, 'trajectory.png'))
    plt.close()
def split_data():
    trajectories = {}
    column_names = None
    with open('../../../Data/4D_Lotka_Volterra/4D_Lotka_T5_N10/trainingp_data50.csv', 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i+1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)
if __name__ == "__main__":
    x1, x2, x3, x4 = generate_random_values()
    initial_conditions = [generate_random_values_based_on_c() for _ in range(5)]  # number of trajectories
    generate_data(initial_conditions)
    data = np.genfromtxt('50.csv', delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 6):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()

Initial data (x1, x2, x3, x4):
(2.4360438789453474, 1.7513145798373926, 2.7607222914244196, 2.3354744373038336)
(0.8506439177035028, 0.9134769261888882, 0.3590223629625286, 0.7185799119599543)
(3.888763777664364, 3.2651381170954683, 0.33044355446955687, 0.7145347825518236)
(0.7759923635288791, 2.456280675181136, 1.434018064029456, 0.9872568338055565)
(2.6144713501508576, 2.9167886257224658, 3.3668736039302662, 3.7118317669983787)


In [9]:
input_directory = r'../../../System/4D_Lotka_Volterra/4D_Lotka_T5_N10'
output_directory = r'../../../Data/4D_Lotka_Volterra/4D_Lotka_T5_N10/Traj_data'
df14 = pd.read_csv(os.path.join(input_directory, '50.csv')) # 50.csv, saved names of the data
Br = pd.read_csv('../../../System/4D_Lotka_Volterra/4D_Lotka_T5_N10/50.csv')
tr = Br.groupby('trajectory').size()
re1 = int(round(tr.mean()))
rows_per_file = re1 
num_files = len(df14) // rows_per_file
data_chunks = [df14.iloc[i * rows_per_file:(i + 1) * rows_per_file].iloc[:, :-1] for i in range(num_files)]
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
for i, chunk in enumerate(data_chunks):
    chunk.to_csv(os.path.join(output_directory, f'tr_{i + 1}.csv'), index=False)